In [2]:
import pandas as pd
import numpy as np
import collections
import itertools
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.layers.core import Activation, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import jieba

Using TensorFlow backend.


### 选取电影评论80000条，进行数据处理，删除无效数据，根据star对数据进行情感分类，star大于等于4，代表喜欢1；否则为不喜欢0。所以本问题，是个二分类问题。

In [3]:
file = pd.read_csv(r'/Users/yang/Desktop/get_douban_comments-master/movie_comments.csv',encoding="utf-8")
file_1=file.iloc[:80000]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#file_1.info()

In [5]:
file_1[file_1['star']=='star'].head()

,id,link,name,comment,star
568,id,link,name,comment,star


In [6]:
file_1.drop([568],inplace=True)
file_1[file_1['star']=='star'].head()

/Users/yang/Library/Python/3.6/lib/python/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,id,link,name,comment,star


In [7]:
file_1['sentiment']=[1 if int(file_1.iloc[i]['star'])>=4 else 0 for i in range(len(file_1))]
file_2=file_1.drop(['link','name','star','id'],axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
#file_2.info

In [9]:
collections.Counter(file_2['sentiment'])

Counter({0: 34691, 1: 45308})

In [13]:
34691/(34691+45308),1-34691/(34691+45308)

(0.4336429205365067, 0.5663570794634933)

### 如上可见，两种类型的标签数量都在50%左右。此问题并非垃圾邮件之类的“偏斜类问题”。可以使用accuracy来进行模型效果衡量。

In [9]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords
stopwords=stopwordslist('/Users/yang/Desktop/stopwords.txt')

comments=[list(jieba.cut(str(i))) for i in file_2['comment']]

def get_corpus(comments,stopwords):
    corpus=[]
    for element in comments:
        sentence=[]
        for word in element:
            if word not in stopwords and (word not in [' ','，','。']):
                sentence.append(word)
        corpus.append(sentence)  
    return corpus
corpus=get_corpus(comments,stopwords)

### 如上，由于电影评论的很多标点符号是英文符号，所以'/Users/yang/Desktop/stopwords.txt'中既有中文停词，又有英文停词。将评论内容进行切词，形成2维的list--comments，以便之后将词替换为数字。对comments中的词进行去停词处理，获得二维的list--corpus。

In [11]:
max_sentence_length=max([(len(i)) for i in comments])
words=list(itertools.chain.from_iterable(corpus))
print('max_length of sentence:',max_sentence_length)
print('number of words:',len(words))
print('number of set(words):',len(set(words)))

max_length of sentence: 159
number of words: 981589
number of set(words): 81488


### 如上， 将corpus由二维list变为1维的list---words，即电影评论中所有词的数量为981589，去除重复之后的词汇量为81488，句子的最大长度为159。

In [12]:
vocabulary_size = 30000

def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return dictionary, reverse_dictionary

word2index, index2word = build_dataset(words)

print(index2word[9])

不错


In [13]:
#for i in range(1000):
#   print(index2word[i])

In [14]:
def get_data(corpus,word2index):
    data = []
    for i in corpus:
        sequence=[]
        for word in i:
            if word in word2index:
                sequence.append(word2index[word])
            else:
                sequence.append(0)
        data.append(sequence)
    return data
data=get_data(corpus,word2index)  

### 如上，构建词汇索引表。
对于words中的81488个词，出现频率最高的前29999个词，按照频率由高到低的次序，编号为1----29999，其余词的编号都为0。生成word2index, index2word字典。然后，根据corpus，生成data--将corpus中的词替换为相应数字编号。

In [15]:
maxlen=100
X=sequence.pad_sequences(data,maxlen=maxlen,padding="pre",value=0.0)

In [16]:
y=file_2['sentiment'].values

In [17]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

### 如上，将句子的长度进行统一处理为100，少的补0；多的截取前面部分。再进行数据集的拆分。

In [18]:
EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE =64
MAX_FEATURES=90000


vocab_size=min(MAX_FEATURES, len(set(words))) + 2

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=maxlen))
model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

### 如上，因为是二分类问题，loss参数选"binary_crossentropy"，Activation选"sigmoid"。
经过对vocabulary_size，dropout，以及评论数量的调整。当评论数量为80000时，最好的结果为vocabulary_size=30000，dropout=0.2。vocabulary_size过大或过小。准确率都会大大降低至50%。猜想原因：vocabulary_size过小，可能会将重要词汇忽略，而过大，将不能识别重要词汇。由于每次训练花费时间较长，效果仍在改进中。

### 判断模型效果的方法：learning rate曲线。
当随着训练样本增多，训练集的准确率很高，测试集的准确率较低，且二者之间有明显差距时，为overfitting。需要减少feature，dropout、regularizarion、早停或者增加训练样本数量。
当随着训练样本增多，训练集的准确率较低，测试集的准确率较低，且二者达到接近时，为underfitting。需要选择新的特征，或者使用更复杂的模型等。


In [19]:
BATCH_SIZE = 64
NUM_EPOCHS = 10
model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))

Train on 63999 samples, validate on 16000 samples
Epoch 1/10
63999/63999 [==============================] - 422s 7ms/step - loss: 0.5444 - acc: 0.7206 - val_loss: 0.4905 - val_acc: 0.7655
Epoch 2/10
63999/63999 [==============================] - 421s 7ms/step - loss: 0.4046 - acc: 0.8145 - val_loss: 0.4916 - val_acc: 0.7722
Epoch 3/10
63999/63999 [==============================] - 420s 7ms/step - loss: 0.3186 - acc: 0.8597 - val_loss: 0.5301 - val_acc: 0.7697
Epoch 4/10
63999/63999 [==============================] - 415s 6ms/step - loss: 0.2507 - acc: 0.8896 - val_loss: 0.6089 - val_acc: 0.7692
Epoch 5/10
63999/63999 [==============================] - 415s 6ms/step - loss: 0.2021 - acc: 0.9091 - val_loss: 0.6963 - val_acc: 0.7693
Epoch 6/10
63999/63999 [==============================] - 415s 6ms/step - loss: 0.1678 - acc: 0.9259 - val_loss: 0.8195 - val_acc: 0.7686
Epoch 7/10
63999/63999 [==============================] - 415s 6ms/step - loss: 0.1418 - acc: 0.9367 - val_loss: 0.8997 - 